In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> # KÜTÜPHANELER 📚

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve
import warnings
warnings.simplefilter(action="ignore")
warnings.filterwarnings("ignore")

# <p style="background-color:#EFCD59;font-family:newtimeroman;color:#F4FBFE;font-size:150%;text-align:center;border-radius:10px 10px;"> DİABETES  </p>

Veri seti ABD'deki Ulusal Diyabet-Sindirim-Böbrek Hastalıkları Enstitüleri'nde tutulan büyük veri setinin parçasıdır. 
ABD'deki Arizona Eyaleti'nin en büyük 5. şehri olan Phoenix şehrinde yaşayan 21 yaş ve üzerinde olan Pima Indian kadınları üzerinde yapılan diyabet araştırması için kullanılan verilerdir. 
768 gözlem ve 8 sayısal bağımsız değişkenden oluşmaktadır.

**DEĞİŞKENLER :**
* Pregnancies: Hamilelik sayısı
* Glucose: Glikoz
* BloodPressure: Kan basıncı (Diastolic(Küçük Tansiyon))
* SkinThickness: Cilt Kalınlığı
* Insulin: İnsülin.
* BMI: Beden kitle indeksi.
* DiabetesPedigreeFunction: Soyumuzdaki kişilere göre diyabet olma ihtimalimizi hesaplayan bir fonksiyon.
* Age: Yaş (yıl)
* Outcome: Kişinin diyabet olup olmadığı bilgisi. Hastalığa sahip (1) ya da değil (0)


In [ ]:
df = pd.read_csv("/kaggle/input/diabetescsv/diabetes.csv")
df.head()

> # GENEL RESMİ İNCELEYELİM

In [ ]:
def check_df(dataframe, head=5):
    print("-------------------- Shape --------------------")
    print(dataframe.shape)
    print("-------------------- İnfo --------------------")
    print(dataframe.info())
    print("-------------------- Types --------------------")
    print(dataframe.dtypes)
    print("-------------------- Head --------------------")
    print(dataframe.head(head))
    print("-------------------- Tail --------------------")
    print(dataframe.tail(head))
    print("-------------------- NA --------------------")
    print(dataframe.isnull().sum())
    print("-------------------- Quantiles --------------------")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

> # NUMERİK VE KATEGORİK DEĞİŞKENLERİN YAKALANMASI

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optional
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.

    """
    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    return cat_cols, num_cols, cat_but_car


cat_cols, num_cols, cat_but_car = grab_col_names(df)

In [ ]:
print(f"kategorik columns : {cat_cols}")

print(f"numerik columns : {num_cols}")

> # KATEGORİK DEĞİŞKENLERİN ANALİZİ

In [ ]:
def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()

In [ ]:
for col in cat_cols:
    cat_summary(df,col,plot=True)

> # NUMERİK DEĞİŞKENLERİN ANALİZİ

In [ ]:
def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)

    if plot:
        dataframe[numerical_col].hist(bins=20)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show()

In [ ]:
for col in num_cols:
    num_summary(df, col, plot=True)

> # NUMERİK DEĞİŞKENLERİN HEDEF DEĞİŞKENE GÖRE ANALİZİ

In [ ]:
def target_summary_with_num(dataframe, target, numerical_col):
    print(dataframe.groupby(target).agg({numerical_col: "mean"}), end="\n\n\n")

for col in num_cols:
    target_summary_with_num(df, "Outcome", col)


> # KORELASYON

In [ ]:
df.corr()

In [ ]:
# Korelasyon Matrisi

f, ax = plt.subplots(figsize=[18, 13])
sns.heatmap(df.corr(), annot=True, fmt=".2f", ax=ax, cmap="magma")
ax.set_title("Correlation Matrix", fontsize=20)
plt.show()

> # BASE MODEL KURULUMU

In [ ]:
y = df["Outcome"]
X = df.drop("Outcome", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=17)

rf_model = RandomForestClassifier(random_state=46).fit(X_train, y_train)
y_pred = rf_model.predict(X_test)


print(f"Accuracy: {round(accuracy_score(y_pred, y_test), 2)}")
print(f"Recall: {round(recall_score(y_pred,y_test),3)}")
print(f"Precision: {round(precision_score(y_pred,y_test), 2)}")
print(f"F1: {round(f1_score(y_pred,y_test), 2)}")
print(f"Auc: {round(roc_auc_score(y_pred,y_test), 2)}")

> # FEATURE ENGİNEERİNG

# EKSİK DEĞER ANALİZİ

Bir insanda Pregnancies ve Outcome dışındaki değişken değerleri 0 olamayacağı bilinmektedir.

Bundan dolayı bu değerlerle ilgili aksiyon kararı alınmalıdır. 

0 olan değerlere NaN atanabilir .


In [ ]:
zero_columns = [col for col in df.columns if (df[col].min() == 0 and col not in ["Pregnancies", "Outcome"])]

zero_columns

In [ ]:
# Gözlem birimlerinde 0 olan degiskenlerin her birisine gidip 0 iceren gozlem degerlerini NaN ile değiştiriyoruz.

for col in zero_columns:
    df[col] = df[col].replace(0, np.nan)

In [ ]:
df.isnull().sum()

In [ ]:
def missing_values_table(dataframe):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

missing_values_table(df)

In [ ]:
# Eksik değerlerin doldurulması:

df = df.apply(lambda x: x.fillna(x.median()), axis=0)
df.isnull().sum()

# AYKIRI DEĞER ANALİZİ

In [ ]:
# AYKIRI DEĞER ANALİZİ FONKSİYONLARI

def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quantile1 = dataframe[col_name].quantile(q1)
    quantile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quantile3 - quantile1
    up_limit = quantile3 + 1.5 * interquantile_range
    low_limit = quantile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
# Aykırı değer sorgulama işlemi

for col in num_cols:
    print(col,check_outlier(df , col))

In [ ]:
# Aykırı değerleri baskılama işlemi

for col in num_cols:
    if check_outlier(df, col):
        replace_with_thresholds(df, col)

for col in num_cols:
    print(col,check_outlier(df , col))

# ÖZELLİK ÇIKARIMI

**BMI**

<= 18.5 : underweight

18.5 - 24.9 : normalweight

25 - 29.9 : overweight

30 <= : obese

In [ ]:
bins = [0, 18.5, 24.9, 29.9, 50]
labels = ['underweight', 'normalweight', 'overweight', 'obese']
df['BMI_GROUP'] = pd.cut(df['BMI'], bins=bins, labels=labels)
df.head()

**AGE**

18 - 29 : young

30 - 49 : adult

50 - 69 : senior

70 <= : elderly


In [ ]:
age_bins =[18,29,49,69,100]
age_labels = ['Young', 'Adult', 'Senior', 'Elderly']
df['AGE_GROUP'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels)
df.head()

In [ ]:
# Glukoz degerini kategorik değişkene çevirme

df["GLUCOSE_GROUP"] = pd.cut(x=df["Glucose"], bins=[0, 140, 200, 300], labels=["Normal", "Prediabetes", "Diabetes"])
df.head()

In [ ]:
# İnsülin değerini kategorik değişkene çevirme

def set_insulin(dataframe, col_name="Insulin"):
    if 16 <= dataframe[col_name] <= 166:
        return "Normal"
    else:
        return "Abnormal"

df["NEW_INSULIN_SCORE"] = df.apply(set_insulin, axis=1)
df.head()

In [ ]:
# Kolon isimlerinin büyütülmesi
df.columns = [col.upper() for col in df.columns]

# ENCODİNG

In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

In [ ]:
df.info()

***LABEL ENCODİNG***

In [ ]:
df["NEW_INSULIN_SCORE"] = LabelEncoder().fit_transform(df["NEW_INSULIN_SCORE"])
df.head()

***ONE - HOT ENCODİNG***

In [ ]:
df = pd.get_dummies(df , columns = ['BMI_GROUP', 'AGE_GROUP', 'GLUCOSE_GROUP'], drop_first = True)
df = df.replace({True: 1, False: 0})
df.head()

# SAYISAL DEĞİŞKENLERİN STANDARTLAŞTIRILMASI

In [ ]:
num_cols

In [ ]:
df[num_cols] = StandardScaler().fit_transform(df[num_cols])

> # MODEL KURMA

In [ ]:
y = df["OUTCOME"]
X = df.drop("OUTCOME", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=17)

rf_model_1 = RandomForestClassifier(random_state=46).fit(X_train, y_train)
y_pred = rf_model_1.predict(X_test)

print(f"Accuracy: {round(accuracy_score(y_pred, y_test), 2)}")
print(f"Recall: {round(recall_score(y_pred,y_test),3)}")
print(f"Precision: {round(precision_score(y_pred,y_test), 2)}")
print(f"F1: {round(f1_score(y_pred,y_test), 2)}")
print(f"Auc: {round(roc_auc_score(y_pred,y_test), 2)}")

In [ ]:
# Hiperparametre optimizasyonu sonrasında model kurma:

rf_model = RandomForestClassifier(random_state=17).fit(X,y)

rf_params = {"max_depth": [5, 8, None],
             "max_features": [3, 5, 7, "auto"],
             "min_samples_split": [5, 8, 15],
             "n_estimators": [100, 200, 400]}


rf_best_grid = GridSearchCV(rf_model, rf_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)

rf_final = rf_model.set_params(**rf_best_grid.best_params_, random_state=17).fit(X, y)

cv_results = cross_validate(rf_final, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc"])

print(cv_results['test_accuracy'].mean())
print(cv_results['test_f1'].mean())
print(cv_results['test_roc_auc'].mean())


# FEATURE İMPORTANCE

In [ ]:
def plot_importance(model, features, num=len(X), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    print(feature_imp.sort_values("Value",ascending=False))
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')

plot_importance(rf_model, X)